In [1]:
import findspark
findspark.init()
from utils.s3_process import  read_key
from utils.s3_process_nlp import read_csv_from_s3, get_latest_s3_object_version
from utils.clean_text import clean_text_column

from utils.mlflow_func import get_latest_model_version, get_model_version_by_stage
import datetime
import yaml
import mlflow
import os 
import time

os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/TranChucThien/kltn-sentiment-monitoring-mlops.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="TranChucThien"


# Thêm các thư viện cần thiết
import nltk
from nltk.corpus import stopwords

from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
# Các import khác...

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col
# 
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param

# 
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import sys
from multiprocessing import Process
import logging

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *

from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/thientran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/thientran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
def create_pipeline():
    
    document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

    tokenizer = Tokenizer() \
        .setInputCols(["document"]) \
        .setOutputCol("token")
        
    normalizer = Normalizer() \
        .setInputCols(["token"]) \
        .setOutputCol("normalized") \
        .setLowercase(True) 
        
    stop_words_cleaner = StopWordsCleaner() \
        .setInputCols(["normalized"]) \
        .setOutputCol("cleanTokens") \
        .setCaseSensitive(False)
    
    lemmatizer = LemmatizerModel.pretrained("lemma_antbnc") \
        .setInputCols(["cleanTokens"]) \
        .setOutputCol("lemmatized") 
            
    word_embeddings_elmo = ElmoEmbeddings.pretrained("elmo", "en") \
        .setInputCols(["document", "lemmatized"]) \
        .setOutputCol("embeddings")

    sentence_embeddings = SentenceEmbeddings() \
        .setInputCols(["document", "embeddings"]) \
        .setOutputCol("sentence_embeddings") \
        .setPoolingStrategy("SUM")  # or "SUM", "MAX"
        
    classifier = ClassifierDLApproach() \
        .setInputCols(["sentence_embeddings"]) \
        .setOutputCol("category") \
        .setLabelColumn("label") \
        .setMaxEpochs(10) \
        .setLr(0.003) \
        .setBatchSize(8) \
        .setEnableOutputLogs(True) \
        .setOutputLogsPath("classifier_logs")
    
    finisher = Finisher() \
        .setInputCols(["category"]) \
        .setOutputCols(["prediction"]) \
        .setCleanAnnotations(False) \
        
        
    pipeline_elmo = Pipeline(stages=[
        document_assembler,
        tokenizer,
        normalizer,
        stop_words_cleaner,
        lemmatizer,
        word_embeddings_elmo,
        sentence_embeddings,
        classifier,
        finisher
    ])
    return pipeline_elmo


def tune_model(pipeline, train_data, use_hashing=True, vectorizer=None, hashingTF=None, lr=None):
    evaluator = MulticlassClassificationEvaluator(labelCol="Label", predictionCol="prediction", metricName="f1")

    paramGrid = ParamGridBuilder()

    if use_hashing:
        paramGrid = paramGrid.addGrid(hashingTF.numFeatures, [1000, 5000, 10000])
    else:
        paramGrid = paramGrid.addGrid(vectorizer.vocabSize, [5000, 10000])

    paramGrid = paramGrid.addGrid(lr.regParam, [0.0, 0.01, 0.1])
    paramGrid = paramGrid.build()

    crossval = CrossValidator(
        estimator=pipeline,
        estimatorParamMaps=paramGrid,
        evaluator=evaluator,
        numFolds=3
    )

    best_model = crossval.fit(train_data)
    
    return best_model


def evaluator(prediction1, label_col="label", prediction_col="prediction"):
    prediction1 = prediction1.withColumn(label_col, col(label_col).cast(DoubleType()))
    prediction1 = prediction1.withColumn(prediction_col, col(prediction_col).cast(DoubleType()))
    evaluator = MulticlassClassificationEvaluator(labelCol=label_col, predictionCol=prediction_col)
    accuracy = evaluator.evaluate(prediction1, {evaluator.metricName: "accuracy"})
    precision = evaluator.evaluate(prediction1, {evaluator.metricName: "weightedPrecision"})
    recall = evaluator.evaluate(prediction1, {evaluator.metricName: "weightedRecall"})
    f1 = evaluator.evaluate(prediction1, {evaluator.metricName: "f1"})

    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-score:  {f1:.4f}")
    return accuracy, precision, recall, f1

def data_distribution(data, label_col="Label"):
    total_count = data.count()
    label_dist = data.groupBy(label_col).count()
    label_dist = label_dist.withColumn("percentage", (col("count") / total_count) * 100)
    label_dist.orderBy(label_col).show()
    return label_dist.orderBy(label_col)

def load_config(config_path="configs/config.yaml"):       
    try:
        with open(config_path, "r") as f:
            config = yaml.safe_load(f)
        return config
    except Exception as e:
        logging.error(f"Failed to load config: {e}")
        raise


def set_up_mlflow_tracking(config, config_secret):
    """Sets up MLflow tracking URI and credentials."""
    os.environ["MLFLOW_TRACKING_PASSWORD"] = config_secret['mlflow']['password']
    os.environ['MLFLOW_TRACKING_URI'] = config['mlflow']['tracking_uri']
    os.environ['MLFLOW_TRACKING_USERNAME'] = config['mlflow']['username']
    mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
    logging.info("MLflow tracking setup complete.")
    
def load_dataset(config, config_secret, spark):
    """Loads dataset from S3."""
    bucket = config['s3']['bucket']
    dataset_key = config['s3']['keys']['dataset']
    
    dataset_path = f"s3a://{bucket}/{dataset_key}"
    logging.info(f"Dataset path: {dataset_path}")
    # AWS credentials and region
    AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY = read_key(config_secret['aws']['access_key_path'])
    AWS_REGION = config['aws']['region']
    S3_OUTPUT_KEY = config['s3']['keys']['dataset']
    BUCKET_NAME = config['s3']['bucket']
    
    logging.info("Reading CSV file from S3...")
    data = read_csv_from_s3(dataset_path, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_REGION, spark)
    logging.info("Read csv file from S3 successfully.")
    data.show(3)
    
    
    return data


def get_data_version(config, config_secret):
    """Loads dataset from S3."""
    bucket = config['s3']['bucket']
    dataset_key = config['s3']['keys']['dataset']
    
    dataset_path = f"s3a://{bucket}/{dataset_key}"
    logging.info(f"Dataset path: {dataset_path}")
    # AWS credentials and region
    AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY = read_key(config_secret['aws']['access_key_path'])
    AWS_REGION = config['aws']['region']
    S3_OUTPUT_KEY = config['s3']['keys']['dataset']
    BUCKET_NAME = config['s3']['bucket']
    
    return get_latest_s3_object_version(s3_path=dataset_path,aws_access_key=AWS_ACCESS_KEY_ID,aws_secret_key=AWS_SECRET_ACCESS_KEY,region=AWS_REGION), dataset_path

def split_data(data, train_ratio=0.8, seed=42):
    train_data, validate_data = data.randomSplit([0.8, 0.2], seed=42)
    logging.info("Data split completed.")
    
    print("Train data:")
    train_data.printSchema()
    train_data.show(3)
    
    print("Validate data:")
    validate_data.printSchema()
    validate_data.show(3)

In [3]:
# Configure logging within this process
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info(f"Starting Text Classification Pipeline for Olmo model...")


# Load configuration
logging.info("Loading configuration from 'configs/config.yaml'")
config = load_config("../configs/config.yaml")
config_secret = load_config("../configs/secrets.yaml")
logging.info("Successfully loaded configuration.")

# Spark session initialization
logging.info("Initializing Spark session...")
spark = sparknlp.start(
    SparkSession.builder \
        .appName("Spark NLP - BERT Sentiment Classification") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.262,com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.driver.maxResultSize", "0") \
        .getOrCreate()
)
logging.info("Spark Session with Spark NLP is ready.")

# Load dataset
logging.info("Loading dataset from S3...")
data = load_dataset(config, config_secret, spark)
data_version, dataset_path = get_data_version(config, config_secret)

data = data.withColumnRenamed("text", "text").withColumnRenamed("label", "label")
data = data.selectExpr("cast(Text as string) as text", "cast(Label as string) as label")
data = data.filter(col("label") != "3")
logging.info("Successfully loaded dataset.")

2025-06-04 10:25:27,344 - INFO - Starting Text Classification Pipeline for Olmo model...
2025-06-04 10:25:27,346 - INFO - Loading configuration from 'configs/config.yaml'
2025-06-04 10:25:27,373 - INFO - Successfully loaded configuration.
2025-06-04 10:25:27,374 - INFO - Initializing Spark session...
your 131072x1 screen size is bogus. expect trouble
25/06/04 10:25:29 WARN Utils: Your hostname, LE11-D5013 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/06/04 10:25:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/thientran/.local/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/thientran/.ivy2/cache
The jars for the packages stored in: /home/thientran/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bb1b2e9b-379b-4930-b6d4-c68a38695a9f;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in spark-list
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#common

25/06/04 10:25:43 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
2025-06-04 10:25:43,291 - INFO - Spark Session with Spark NLP is ready.
2025-06-04 10:25:43,292 - INFO - Loading dataset from S3...
2025-06-04 10:25:43,293 - INFO - Dataset path: s3a://tranchucthien-bucket/dataset/dataset.csv
2025-06-04 10:25:43,495 - INFO - Reading CSV file from S3...
25/06/04 10:25:43 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
2025-06-04 10:26:03,581 - INFO - Read csv file from S3 successfully.            
2025-06-04 10:26:05,089 - INFO - Dataset path: s3a://tranchucthien-bucket/dataset/dataset.csv


+-----+--------------------+
|Label|                Text|
+-----+--------------------+
|    1|im getting border...|
|    1|im coming borderl...|
|    1|im getting border...|
+-----+--------------------+
only showing top 3 rows



2025-06-04 10:26:06,597 - INFO - Successfully loaded dataset.


In [4]:
logging.info("Setting up MLflow experiment...")
set_up_mlflow_tracking(config=config, config_secret=config_secret)
experiment_name = f'DL_Elmo_Text_Classification_Experiment'
mlflow.set_experiment(experiment_name)
logging.info("MLflow experiment set up successfully with name: %s", experiment_name)

2025-06-04 10:26:06,607 - INFO - Setting up MLflow experiment...
2025-06-04 10:26:06,610 - INFO - MLflow tracking setup complete.
2025-06-04 10:26:08,817 - INFO - MLflow experiment set up successfully with name: DL_Elmo_Text_Classification_Experiment


In [5]:
total_count = data.count()
logging.info(f"Total samples: {data.count()}")

2025-06-04 10:26:20,265 - INFO - Total samples: 59497                           


In [6]:
from functools import reduce
def k_fold_split(data, k=3, seed=42):
    # Chia dữ liệu thành k phần bằng randomSplit
    weights = [1.0 / k] * k
    return data.randomSplit(weights, seed=seed)

def cross_validate_custom(data, pipeline, k=3):
    folds = k_fold_split(data, k)
    metrics = []

    for i in range(k):
        logging.info(f"=== Fold {i + 1}/{k} ===")
        validate_data = folds[i]
        # Lấy tất cả các fold ngoại trừ fold[i] làm train
        train_folds = [folds[j] for j in range(k) if j != i]
        train_data = reduce(lambda df1, df2: df1.union(df2), train_folds)

        model = pipeline.fit(train_data)
        prediction = model.transform(validate_data).withColumn("prediction", col("prediction")[0].cast("string"))
        accuracy, precision, recall, f1 = evaluator(prediction, label_col="label", prediction_col="prediction")
        metrics.append((accuracy, precision, recall, f1))

    return metrics


In [13]:
pipeline = create_pipeline()

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
elmo download started this may take some time.
Approximate size to download 334.1 MB
[OK!]


In [ ]:
train_data, validate_data = data.randomSplit([0.9, 0.1], seed=42)
a,b =  validate_data.randomSplit([0.9, 0.1], seed=42)
print("Count train data:", b.count())

In [14]:
metrics = cross_validate_custom(data, pipeline, k=5)  # hoặc k=5 tùy bạn


2025-06-04 14:10:16,319 - INFO - === Fold 1/5 ===
2025-06-04 14:15:54.260265: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/9c16db13d6b4_classifier_dl9842502838296282843
2025-06-04 14:15:55.096785: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2025-06-04 14:15:55.096949: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/9c16db13d6b4_classifier_dl9842502838296282843
2025-06-04 14:15:56.114249: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2025-06-04 14:15:57.157219: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/9c16db13d6b4_classifier_dl9842502838296282843
2025-06-04 14:15:57.311500: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { ser

Training started - epochs: 10 - learning_rate: 0.003 - batch_size: 8 - training_examples: 47528 - classes: 3
Epoch 1/10 - 27.69s - loss: 7199.6724 - acc: 0.33767465 - batches: 5941
Epoch 2/10 - 27.50s - loss: 5642.485 - acc: 0.60122454 - batches: 5941
Epoch 3/10 - 27.71s - loss: 4953.927 - acc: 0.7173666 - batches: 5941
Epoch 4/10 - 30.40s - loss: 4633.6206 - acc: 0.7736282 - batches: 5941
Epoch 5/10 - 31.90s - loss: 4417.34 - acc: 0.8114585 - batches: 5941
Epoch 6/10 - 33.83s - loss: 4257.999 - acc: 0.83599144 - batches: 5941
Epoch 7/10 - 32.96s - loss: 4153.5117 - acc: 0.85490656 - batches: 5941
Epoch 8/10 - 35.45s - loss: 4081.1377 - acc: 0.86744654 - batches: 5941
Epoch 9/10 - 30.51s - loss: 4033.9739 - acc: 0.8763045 - batches: 5941
Epoch 10/10 - 30.55s - loss: 3999.4333 - acc: 0.8826797 - batches: 5941


2025-06-04 14:28:46,934 - INFO - === Fold 2/5 ===                               


Accuracy:  0.7952
Precision: 0.7953
Recall:    0.7952
F1-score:  0.7942


2025-06-04 14:34:12.560927: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/4fce709856d1_classifier_dl12574968973225978250
2025-06-04 14:34:12.693493: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2025-06-04 14:34:12.693566: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/4fce709856d1_classifier_dl12574968973225978250
2025-06-04 14:34:13.185148: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2025-06-04 14:34:14.255207: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/4fce709856d1_classifier_dl12574968973225978250
2025-06-04 14:34:14.430018: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 1869118 microse

...

2025-06-04 15:25:17,113 - INFO - === Fold 5/5 ===                               


Accuracy:  0.2991
Precision: 0.0895
Recall:    0.2991
F1-score:  0.1377


2025-06-04 15:30:35.951814: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/42a07aedb599_classifier_dl15316262005986145512
2025-06-04 15:30:36.065159: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2025-06-04 15:30:36.065228: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/42a07aedb599_classifier_dl15316262005986145512
2025-06-04 15:30:36.505904: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2025-06-04 15:30:37.273898: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/42a07aedb599_classifier_dl15316262005986145512
2025-06-04 15:30:37.421947: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 1470142 microse

Training started - epochs: 10 - learning_rate: 0.003 - batch_size: 8 - training_examples: 47602 - classes: 3
Epoch 1/10 - 27.76s - loss: 7087.7466 - acc: 0.35737395 - batches: 5951
Epoch 2/10 - 27.26s - loss: 5273.9336 - acc: 0.6487605 - batches: 5951
Epoch 3/10 - 28.62s - loss: 4879.205 - acc: 0.7222059 - batches: 5951
Epoch 4/10 - 29.95s - loss: 4560.129 - acc: 0.7771429 - batches: 5951
Epoch 5/10 - 29.22s - loss: 4358.7744 - acc: 0.81334037 - batches: 5951
Epoch 6/10 - 29.64s - loss: 4205.1816 - acc: 0.8366386 - batches: 5951
Epoch 7/10 - 31.39s - loss: 4121.002 - acc: 0.8532983 - batches: 5951
Epoch 8/10 - 32.91s - loss: 4058.8323 - acc: 0.8655672 - batches: 5951
Epoch 9/10 - 30.35s - loss: 4002.6511 - acc: 0.8752521 - batches: 5951
Epoch 10/10 - 30.25s - loss: 3959.205 - acc: 0.8821008 - batches: 5951


Accuracy:  0.7938
Precision: 0.7943
Recall:    0.7938
F1-score:  0.7932


In [11]:
avg_accuracy = sum([m[0] for m in metrics]) / len(metrics)
avg_precision = sum([m[1] for m in metrics]) / len(metrics)
avg_recall = sum([m[2] for m in metrics]) / len(metrics)
avg_f1 = sum([m[3] for m in metrics]) / len(metrics)

print(f"Average Accuracy: {avg_accuracy:.4f}")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1-score: {avg_f1:.4f}")

Average Accuracy: 0.8028
Average Precision: 0.8029
Average Recall: 0.8028
Average F1-score: 0.8023


In [5]:
from functools import reduce

def cross_validate_custom(data, pipeline, k=3):
    folds = k_fold_split(data, k)
    metrics = []

    for i in range(k):
        logging.info(f"=== Fold {i + 1}/{k} ===")
        validate_data = folds[i]
        train_folds = [folds[j] for j in range(k) if j != i]
        train_data = reduce(lambda df1, df2: df1.union(df2), train_folds)

        model = pipeline.fit(train_data)
        prediction = model.transform(validate_data).withColumn("prediction", col("prediction")[0].cast("string"))
        accuracy, precision, recall, f1 = evaluator(prediction, label_col="label", prediction_col="prediction")
        metrics.append((accuracy, precision, recall, f1))

    return metrics


def run_grid_search_cv(data, k_fold=3):
    lr_list = [0.003, 0.01]
    batch_size_list = [8]
    epoch_list = [5]

    best_f1 = 0
    best_params = {}

    for lr in lr_list:
        for batch_size in batch_size_list:
            for epochs in epoch_list:
                logging.info(f"Running CV for lr={lr}, batch_size={batch_size}, epochs={epochs}")

                classifier = ClassifierDLApproach() \
                    .setInputCols(["sentence_embeddings"]) \
                    .setOutputCol("category") \
                    .setLabelColumn("label") \
                    .setMaxEpochs(epochs) \
                    .setLr(lr) \
                    .setBatchSize(batch_size) \
                    .setEnableOutputLogs(False)

                pipeline = Pipeline(stages=[
                    DocumentAssembler().setInputCol("text").setOutputCol("document"),
                    Tokenizer().setInputCols(["document"]).setOutputCol("token"),
                    Normalizer().setInputCols(["token"]).setOutputCol("normalized").setLowercase(True),
                    StopWordsCleaner().setInputCols(["normalized"]).setOutputCol("cleanTokens").setCaseSensitive(False),
                    LemmatizerModel.pretrained("lemma_antbnc").setInputCols(["cleanTokens"]).setOutputCol("lemmatized"),
                    ElmoEmbeddings.pretrained("elmo", "en").setInputCols(["document", "lemmatized"]).setOutputCol("embeddings"),
                    SentenceEmbeddings().setInputCols(["document", "embeddings"]).setOutputCol("sentence_embeddings").setPoolingStrategy("AVERAGE"),
                    classifier,
                    Finisher().setInputCols(["category"]).setOutputCols(["prediction"]).setCleanAnnotations(False)
                ])

                with mlflow.start_run(run_name=f"CV_lr{lr}_bs{batch_size}_ep{epochs}") as run:
                    metrics = cross_validate_custom(data, pipeline, k=k_fold)
                    avg_accuracy = sum([m[0] for m in metrics]) / len(metrics)
                    avg_precision = sum([m[1] for m in metrics]) / len(metrics)
                    avg_recall = sum([m[2] for m in metrics]) / len(metrics)
                    avg_f1 = sum([m[3] for m in metrics]) / len(metrics)

                    mlflow.log_param("lr", lr)
                    mlflow.log_param("batch_size", batch_size)
                    mlflow.log_param("maxEpochs", epochs)
                    mlflow.log_param("k_fold", k_fold)

                    mlflow.log_metric("avg_accuracy", avg_accuracy)
                    mlflow.log_metric("avg_precision", avg_precision)
                    mlflow.log_metric("avg_recall", avg_recall)
                    mlflow.log_metric("avg_f1", avg_f1)

                    if avg_f1 > best_f1:
                        best_f1 = avg_f1
                        best_params = {"lr": lr, "batch_size": batch_size, "epochs": epochs}
                        logging.info(f"New best F1: {best_f1:.4f} with {best_params}")

    logging.info(f"Best result: F1={best_f1:.4f} with {best_params}")
    return best_params, best_f1

In [10]:
best_params, best_f1 = run_grid_search_cv(data, k_fold=3)

2025-06-03 10:22:13,472 - INFO - Running CV for lr=0.003, batch_size=8, epochs=5


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
elmo download started this may take some time.
Approximate size to download 334.1 MB
[OK!]


2025-06-03 10:22:21,883 - INFO - === Fold 1/3 ===
2025-06-03 10:27:26.782891: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/8849228df1f7_classifier_dl1773962856390756356
2025-06-03 10:27:26.840734: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2025-06-03 10:27:26.840815: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/8849228df1f7_classifier_dl1773962856390756356
2025-06-03 10:27:27.340335: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2025-06-03 10:27:28.571469: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/8849228df1f7_classifier_dl1773962856390756356
2025-06-03 10:27:28.756151: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { ser

Training started - epochs: 5 - learning_rate: 0.003 - batch_size: 8 - training_examples: 39687 - classes: 3
Epoch 1/5 - 23.90s - loss: 4492.515 - acc: 0.6418311 - batches: 4961
Epoch 2/5 - 23.36s - loss: 4153.958 - acc: 0.7221486 - batches: 4961
Epoch 3/5 - 22.92s - loss: 3917.5547 - acc: 0.7716446 - batches: 4961
Epoch 4/5 - 24.68s - loss: 3751.4656 - acc: 0.80357504 - batches: 4961
Epoch 5/5 - 27.88s - loss: 3636.135 - acc: 0.8250972 - batches: 4961


2025-06-03 10:42:35,005 - INFO - === Fold 2/3 ===                               


...

Accuracy:  0.3644
Precision: 0.1328
Recall:    0.3644
F1-score:  0.1946


2025-06-03 12:10:36.070251: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/63b80be1792f_classifier_dl13265948584870888484
2025-06-03 12:10:36.196035: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2025-06-03 12:10:36.196258: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/63b80be1792f_classifier_dl13265948584870888484
2025-06-03 12:10:36.955887: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2025-06-03 12:10:37.959790: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/63b80be1792f_classifier_dl13265948584870888484
2025-06-03 12:10:38.177603: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 2107361 microse

Training started - epochs: 5 - learning_rate: 0.01 - batch_size: 8 - training_examples: 39841 - classes: 3
Epoch 1/5 - 23.86s - loss: 5875.1333 - acc: 0.36666667 - batches: 4981
Epoch 2/5 - 23.50s - loss: 5874.5596 - acc: 0.36666667 - batches: 4981
Epoch 3/5 - 23.31s - loss: 5874.5596 - acc: 0.36666667 - batches: 4981
Epoch 4/5 - 22.53s - loss: 5874.5596 - acc: 0.36666667 - batches: 4981
Epoch 5/5 - 24.53s - loss: 5874.5596 - acc: 0.36666667 - batches: 4981


Accuracy:  0.3679
Precision: 0.1353
Recall:    0.3679
F1-score:  0.1979
🏃 View run CV_lr0.01_bs8_ep5 at: https://dagshub.com/TranChucThien/kltn-sentiment-monitoring-mlops.mlflow/#/experiments/63/runs/ef9fbf47519f413c954ee11f8fc9cc93
🧪 View experiment at: https://dagshub.com/TranChucThien/kltn-sentiment-monitoring-mlops.mlflow/#/experiments/63


2025-06-03 12:25:26,072 - INFO - Best result: F1=0.7571 with {'lr': 0.003, 'batch_size': 8, 'epochs': 5}


In [11]:
print(best_params)
print(best_f1)

{'lr': 0.003, 'batch_size': 8, 'epochs': 5}
0.7571472894732002
